pandas的merge相当于MySQL的join，根据key将不同的df合并在一起

- 理解一对一、一对多、多对多
    - 1 VS 1: 数量关系是 1*1
    - 1 VS N: 数量关系是 1*N
    - M VS N: 数量关系是 M*N
- left join、right join、inner join、outer join的区别
    - left join：取left的值，包括 left & right
    - right join: 取right的值，包括 left & right
    - inner join：取left & right 的交集
    - outer join：取left | right 的并集
- 出现非key字段重合如何处理？

In [1]:
import pandas as pd

In [2]:
# 读取ratting数据

In [3]:
df_rattings=pd.read_csv('./data/ml-25m/ratings.csv')

In [7]:
df_rattings.head(5)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [8]:
df_movies=pd.read_csv('./data/ml-25m/movies.csv')

In [9]:
df_movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
# 将movies 和 ratting信息merge
df_movies_rattings=pd.merge(df_movies, df_rattings, left_on="movieId", right_on="movieId",how="inner")

In [16]:
df_movies_rattings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,1141415820
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0,1573944252
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,858625949
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,890492517


##### 演示 1v1、1vN、M*N关系

In [18]:
left=pd.DataFrame({
    "学号":["1001","1002","1003"],
    "姓名": ["张三","李四", "王五"]
})
right = pd.DataFrame({
    "学号":["1001","1002","1003"],
    "爱好":["打球","学习","跳舞"]
})

In [19]:
# 1 VS 1
pd.merge(left,right,left_on="学号",right_on="学号", how="inner")

,学号,姓名,爱好
0,1001,张三,打球
1,1002,李四,学习
2,1003,王五,跳舞


In [26]:
right = pd.DataFrame({
    "学号":["1001","1001","1002","1003"],
    "爱好":["打球","踢球","学习","跳舞"]
})

In [27]:
# 1 VS N
pd.merge(left,right,left_on="学号",right_on="学号", how="inner")

,学号,姓名,爱好
0,1001,张三,打球
1,1001,张三,踢球
2,1001,小张,打球
3,1001,小张,踢球
4,1002,李四,学习
5,1003,王五,跳舞


In [28]:
left=pd.DataFrame({
    "学号":["1001","1001","1002","1003"],
    "姓名": ["张三","小张","李四", "王五"]
})

In [29]:
# M * N
pd.merge(left,right,left_on="学号",right_on="学号", how="inner")

,学号,姓名,爱好
0,1001,张三,打球
1,1001,张三,踢球
2,1001,小张,打球
3,1001,小张,踢球
4,1002,李四,学习
5,1003,王五,跳舞


##### left join、right join 、inner join、full join

In [42]:
left=pd.DataFrame({
    "id":[1,2,3],
    "col1": ["A","B","C"],\
    
})
right=pd.DataFrame({
    "col2": ["AA","BB","DD"],
    "id":[1,2,4]
})


In [43]:
pd.merge(left,right,how="left")

,id,col1,col2
0,1,A,AA
1,2,B,BB
2,3,C,NaN


In [44]:
pd.merge(left,right,how="right")

,id,col1,col2
0,1,A,AA
1,2,B,BB
2,4,NaN,DD


In [45]:
pd.merge(left,right,how="inner")

,id,col1,col2
0,1,A,AA
1,2,B,BB


In [46]:
pd.merge(left,right,how="outer")

,id,col1,col2
0,1,A,AA
1,2,B,BB
2,3,C,NaN
3,4,NaN,DD


#### 出现非key字段重合如何处理？

In [47]:
left=pd.DataFrame({
    "id":[1,2,3,4],
    "name":["张三","李四","王五","徐柳"],
    "age":[10,20,30,40]
})
right=pd.DataFrame({
    "id":[1,2,3,4],
    "name":["张三2","李四2","王五2","徐柳2"],
    "gender":[1,0,1,1]
})

In [49]:
# 默认不指定on的话会去掉right的重名key
pd.merge(left,right)

,id,name,age,gender
0,1,张三,10,1
1,2,李四,20,0
2,3,王五,30,1
3,4,徐柳,40,1


In [52]:
# 指定on的话，会在重名的字段后缀加上_x、_y等
pd.merge(left,right,on="id")

,id,name_x,age,name_y,gender
0,1,张三,10,张三,1
1,2,李四,20,李四,0
2,3,王五,30,王五,1
3,4,徐柳,40,徐柳,1


In [ ]:
# 指定on的话, 我们可以指定
pd.merge(left,right,on="id", suffixes=["_a","_b"])
